### Numerical Intergration
#### Author: Alina Ozhegova
Date: 07/06/2019

In [200]:
import requests
from IPython.display import Image
import numpy as np
from scipy import stats as sts
from scipy import optimize as opt
from scipy import integrate

### Problem 2.1

In [111]:
def num_integration(g, a, b, N, method):
    area_sum = 0
    #Midpoint rule
    if method == 'midpoint':
        x = np.zeros(N, dtype = np.float64)
        for i in range(N):
            x[i] = a + (2 * i + 1) * (b - a)/(2 * N)
            area_sum = area_sum + g(x[i])
        area_sum = area_sum * (b - a)/N
    #Trapezoid rule
    elif method == 'trapezoid':
        area_sum = 0
        x = np.zeros(N + 1, dtype = np.float64)
        for i in range(N + 1):
            x[i] = a + i * (b - a)/N
            if i == 0:
                area_sum = area_sum + g(x[i])
            elif i == N:
                area_sum = area_sum + g(x[i])
            else:
                area_sum = area_sum + 2 * g(x[i])
        area_sum = ((b - a)/(2 * N)) * area_sum    
    #Simpson's rule
    elif method == 'Simpsons':
        x = np.zeros(2 * N + 1, dtype = np.float64)
        for i in range(2 * N + 1):
            x[i] = a + i * (b - a)/(2 * N)
            if i == 0:
                area_sum = area_sum + g(x[i])
            elif i == 2 * N:
                area_sum = area_sum + g(x[i])
            elif i % 2 == 1:
                area_sum = area_sum + 4 * g(x[i])
            elif i % 2 == 0:
                area_sum = area_sum + 2 * g(x[i])
        area_sum = area_sum * (b - a) / (3 * (2 * N + 1))
    return area_sum

In [112]:
#Check
def g(x):
    return .1 * x ** 4 - 1.5 * x ** 3 + .53 * x ** 2 + 2 * x + 1

In [113]:
res = num_integration(g, -10, 10, 10000, method = 'Simpsons')
print(res)

4373.1146776


### Problem 2.2

In [140]:
def NormalDistApprox(m, sd, N, k):
    z = np.linspace(m - k * sd, m + sd * k, N)
    w = np.zeros(N, dtype = np.float64)
    for i in range(1, N + 1):
        if i == 1:
            w[i - 1] = sts.norm.cdf((z[i - 1] + z[i])/2, loc = m, scale = sd)
        elif i == N:
            w[i - 1] = 1 - sts.norm.cdf((z[N - 2] + z[N - 1])/2, loc = m, \
                                    scale = sd)
        else:
            w[i - 1] = sts.norm.pdf(z[i - 1], loc = m, scale = sd) \
            * (z[N - 1] - z[0])/N
    return w, z

In [146]:
w, z = NormalDistApprox(0, 1, 11, 5)
print(w)
print(z)

[  3.39767312e-06   1.21663842e-04   4.02895310e-03   4.90826968e-02
   2.19973386e-01   3.62674800e-01   2.19973386e-01   4.90826968e-02
   4.02895310e-03   1.21663842e-04   3.39767312e-06]
[-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]


### Problem 2.3

In [147]:
def LogNormalApprox(m , sd, N, k):
    w, z = NormalDistApprox(m , sd, N, k)
    A = np.exp(z)
    return w, A

In [148]:
w, A = LogNormalApprox(0, 1, 11, 5)
print(w)
print(A)

[  3.39767312e-06   1.21663842e-04   4.02895310e-03   4.90826968e-02
   2.19973386e-01   3.62674800e-01   2.19973386e-01   4.90826968e-02
   4.02895310e-03   1.21663842e-04   3.39767312e-06]
[  6.73794700e-03   1.83156389e-02   4.97870684e-02   1.35335283e-01
   3.67879441e-01   1.00000000e+00   2.71828183e+00   7.38905610e+00
   2.00855369e+01   5.45981500e+01   1.48413159e+02]


### Problem 2.4

In [152]:
m, sd, N, k = 10.5, .8, 1000, 5

In [153]:
w, A = LogNormalApprox(m, sd, N, k)

In [158]:
AvIncome = np.average(A, weights = w)
print(AvIncome)#very close

50010.9885932


In [160]:
TrueValue = np.exp(m + (sd**2)/2)
print(TrueValue)

50011.0870085


### Problem 3.1

In [197]:
def GaussQuadApprox(input):
    input = w1, w2, w3, x1, x2, x3
    r = np.zeros(6, dtype=np.float64)
    res = np.zeros(6, dtype = np.float64)
    for i in range(6):
        r[i] = (1 ** (i + 1) - (-1) ** (i + 1))/(i + 1)
    for j in range(6):
        res[j] = np.abs(r[i] - w1 * (x1 ** j) - w2 * (x2 ** j) - \
                        w3 * (x3 ** j))
    return res

In [198]:
input = np.array([1, 1, 1, - .5, 0, 0.5])
solution = opt.fsolve(GaussQuadApprox, input)
solution

/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


array([ 1. ,  1. ,  1. , -0.5,  0. ,  0.5])

In [195]:
def g(x):
    return .1 * x ** 4 - 1.5 * x ** 3 + .53 * x ** 2 + 2 * x + 1

In [196]:
np.sum(solution[:3] * g(solution[3:]))

3.2774999999999999

In [189]:
def system_errors(x):
    w1, w2, w3, x1, x2, x3 = x
    r = np.zeros(6, dtype=np.float64)
    err = np.zeros(6, dtype = np.float64)
    for i in range(6):
        r[i] = (1 ** (i + 1) - (-1) ** (i + 1))/(i + 1)
    
    for ii in range(6):
        err[ii] = np.abs(rhs[ii] - w1 * (x1 ** ii) - w2 * (x2 ** ii) - w3 * (x3 ** ii))
    return err

In [190]:
guess = np.array([1, 1, 1, -.5, 0, .5])
system_errors(guess)

array([ 1.        ,  0.        ,  0.16666667,  0.        ,  0.275     ,  0.        ])

### Exercise 3.2

In [210]:
sol = integrate.quad(g, -10, 10)
value = sol[0]
error = sol[1]
print(value)
print(error)

4373.333333333334
8.109531705284936e-11


### Exercise 4.1

In [226]:
np.random.seed = 25

In [227]:
def MonteCarloApprox(g, x, omega, N):
    pi = 0
    n = len(x)
    area = np.prod(omega[:,1] - omega[:,0])
    for ii in range(N):
        draws = np.random.rand(n) * (omega[:, 1] - omega[:,0]) + omega[:,0]
        pi = pi + g(draws.T)
    pi = area * (1/N) * pi
    
    return pi

In [244]:
circle_area = lambda x: x[0] ** 2 + x[1] ** 2

In [245]:
def g(x):
    if circle_area(x) <= 1:
        res = 1
    else:
        res = 0
    return res

In [246]:
omega = np.array([[-1, 1], [-1, 1]])

In [249]:
MonteCarloApprox(g, np.array([0,0]), omega, 100)

2.8799999999999999